In [1]:
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

### Main Chart

In [2]:
# New orders for capital goods excluding defense or aircraft
key = census_key
base = 'https://api.census.gov/data/timeseries/eits/marts/'
param = 'cell_value,time_slot_id,category_code'
t = '&time=from+1992'
dtc = '&data_type_code=SM'
oth = '&for=us&seasonally_adj=yes'

url = f'{base}?get={param}&key={key}{dtc}{t}{oth}'

r = requests.get(url).json()

In [3]:
df = pd.DataFrame()
for series in ['44000', '44X72', '44W72', '454']:
    df[series] = pd.Series(
        {pd.to_datetime(i[4]): 
         float(i[0]) for i in r[1:] if i[2] == series}
    ).sort_index()
    
data = (df.pct_change(12) * 100).dropna()
data['NS_3M'] = data['454'].rolling(3).mean()

s = ['A191RC']

gdp = nipa_df(retrieve_table('T10105')['Data'], s)

data.to_csv(data_dir / 'marts.csv', index_label='date', float_format='%g')

node = end_node(data['NS_3M'], 'blue!70!black')
write_txt(text_dir / 'rs_ns3m_node.txt', node)

node = end_node(data['44X72'], 'green!90!blue')
write_txt(text_dir / 'rs_44x72_node.txt', node)

totval = df['44X72'].iloc[-1]
shgdp = (totval * 12 / gdp.iloc[-1] * 100)[-1]
totgr = data['44X72'].iloc[-1]
datelt = df.index[-1].strftime('%B %Y')
grtxt = inc_dec_percent(totgr)
    
totval2 = df['454'].iloc[-1]
shgdp2 = (totval2 * 12 / gdp.iloc[-1] * 100)[-1]
totgr2 = data['454'].iloc[-1]
grtxt2 = inc_dec_percent(totgr2)
    
text = ('According to the \href{https://www.census.gov/retail/index.html}{Census Bureau}, '+
        'retail and food service '+
        f'sales totalled \${totval/1000:,.1f} billion in {datelt}, equivalent '+
        f'to roughly {shgdp:.1f} percent of GDP on an annualized basis. '+
        'Over the past year, retail and '+
        f'food service sales {grtxt}, without adjusting for prices '+
        '(see {\color{green!90!blue}\\textbf{---}}). Nonstore '+
        f'sales, which include online retailers, have {grtxt2} over '+
        'the same period (see {\color{blue!70!black}\\textbf{---}}), and '
        f'total \${totval2/1000:,.1f} billion, or roughly {shgdp2:.1f} percent of GDP. ')

write_txt(text_dir / 'marts.txt', text)

text

'According to the \\href{https://www.census.gov/retail/index.html}{Census Bureau}, retail and food service sales totalled \\$485.5 billion in May 2020, equivalent to roughly 27.1 percent of GDP on an annualized basis. Over the past year, retail and food service sales decreased by 6.1 percent, without adjusting for prices (see {\\color{green!90!blue}\\textbf{---}}). Nonstore sales, which include online retailers, have increased by 30.8 percent over the same period (see {\\color{blue!70!black}\\textbf{---}}), and total \\$86.4 billion, or roughly 4.8 percent of GDP. '

### By Type

In [4]:
key = census_key
base = 'https://api.census.gov/data/timeseries/eits/marts/'
param = 'cell_value,time_slot_id,category_code'
t = '&time=from+1992'
dtc = '&data_type_code=SM'
oth = '&for=us&seasonally_adj=no'

url = f'{base}?get={param}&key={key}{dtc}{t}{oth}'

r = requests.get(url).json()

In [5]:
series_dict = {'441': 'Motor Vehicles \& Parts', '442': 'Furniture \& Home Furnishings', 
               '443': 'Electronics \& Appliance', '444': 'Building \& Garden Equipment', 
               '445': 'Food \& Beverage Stores', '446': 'Health \& Personal Care', 
               '447': 'Gasoline Stations', '448': 'Clothing and Accessories', 
               '451': 'Sports/Hobby/Music/Books', '452': 'General Merchandise', 
               '454': 'Nonstore', '722': 'Food Service \& Drinking Places'}

df = pd.DataFrame()
for series in series_dict.keys():
    df[series] = pd.Series(
        {pd.to_datetime(i[4]): 
         float(i[0]) for i in r[1:] if i[2] == series}
    ).sort_index()
    
dpi = fred_df('DSPI', start='1988')['VALUE'] * 1000
data = (df.multiply(12).divide(dpi, axis=0) * 100).dropna()
results = (pd.concat([data.iloc[-61], data.iloc[-1]], axis=1)
             .sort_values(data.dropna().index[-1], ascending=False))
results.index = results.index.map(series_dict)
ltmonth = dtxt(data.dropna().index[-1])['mon2']
write_txt(text_dir / 'rs_mon1.txt', ltmonth)
prmonth = dtxt(data.dropna().index[-61])['mon2']
write_txt(text_dir / 'rs_mon2.txt', prmonth)
results.round(2).to_csv(data_dir / 'rs_comp.csv', index_label='name')

### Inventories to Sales Ratio (ISRATIO)

In [4]:
# Nominal
df = fred_df('ISRATIO')['VALUE']
df.to_csv(data_dir / 'isratio.csv', index_label='date')

# Real 
years = ','.join(map(str, range(1988, 2021)))
url = (f'https://www.bea.gov/api/data/?&UserID={bea_key}'+
        '&method=GetData&datasetname=NIUnderlyingDetail&TableName=U003BU'+
       f'&LineNumber=1&Frequency=M&Year={years}&ResultFormat=json')

r = requests.get(url).json()

data = r['BEAAPI']['Results']['Data']

s1 = pd.read_csv('raw/real_isratio.csv')
s1['Date'] = pd.to_datetime(s1['Date'])
s1 = s1.set_index('Date')['Value'].sort_index()

s2 = pd.Series({pd.to_datetime(i['TimePeriod'], format='%YM%m'): 
               i['DataValue'] for i in data if i['LineNumber'] == '1'}).astype(float)
s2.index.name = 'Date'
s2.name = 'Value'
s2 = s2.sort_index()

result = s1.append(s2)

result.to_csv(data_dir / 'real_isratio.csv', index_label='date')

# Text
ltmon = dtxt(df.index[-1])['mon1']
prmon = dtxt(df.index[-2])['mon1']
prmon12 = dtxt(df.index[-13])['mon1']
ltval = f'{df.iloc[-1]:.2f}'
prval = f'{df.iloc[-2]:.2f}'
prval12 = f'{df.iloc[-13]:.2f}'
lt2mon = dtxt(result.index[-1])['mon1']
pr2mon = dtxt(result.index[-13])['mon1']
lt2val = f'{result.iloc[-1]:.2f}'
lt2val2 = f'{result.iloc[-2]:.2f}'
rngval = f'{result.loc["2011":"2014"].mean():.2f}'

text = (f'In {ltmon}, the ratio of inventories to '+
        f'sales was {ltval}, compared to {prval} in '+
        f'{prmon}, and {prval12} in {prmon12}. The '+
        'inflation adjusted version from BEA shows '+
        f'inventories at {lt2val} times sales in '+
        f'{lt2mon}, compared to {lt2val2} in {pr2mon}. '+
        f'From 2011 to 2014, real monthly inventories were {rngval} '+
         'times real monthly sales, on average. ')

write_txt(text_dir / 'isratio.txt', text)

text

KeyError: 'Results'